# Basic damage detection in Wikipedia
This notebook demonstrates the basic contruction of a vandalism classification system using the [revscoring](http://pythonhosted.org/revscoring/) library that we have developed specifically for classification models of MediaWiki stuff.

The basic process that we'll follow is this:

1. Gather example of human judgement applied to Wikipedia edits.  In this case, we'll take advantage of [reverts](https://meta.wikimedia.org/wiki/Research:Revert).  
2. Split the data into a training and testing set
3. Training the machine learning model
4. Testing the machine learning model

And then we'll have some fun applying the model to some edits using RCStream.  The following diagram gives a good sense for the whole process of training and evaluating a model.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/Supervised_machine_learning_in_a_nutshell.svg/640px-Supervised_machine_learning_in_a_nutshell.svg.png" /></center>

## Part 1: Getting labeled observations
<img style="float: right; margin: 1ex;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/0f/Machine_learning_nutshell_--_Gather_labeled_observations.svg/300px-Machine_learning_nutshell_--_Gather_labeled_observations.svg.png" />

Regretfully, running SQL queries isn't something we can do directly from the notebook *yet*.  So, we'll use [Quarry](https://quarry.wmflabs.org) to generate a nice random sample of edits.  20,000 observations should do just fine.  Here's the query I want to run:

```SQL
USE enwiki_p;
SELECT rev_id 
FROM revision 
WHERE rev_timestamp BETWEEN "20150201" AND "20160201" 
ORDER BY RAND() 
LIMIT 20000;
```

See http://quarry.wmflabs.org/query/7530.  By clicking around the UI, I can see that this URL will download my tab-separated file: http://quarry.wmflabs.org/run/65415/output/0/tsv?download=true

In [2]:
# Magical ipython notebook stuff puts the result of this command into a variable
revids_f = !wget http://quarry.wmflabs.org/run/65415/output/0/tsv?download=true -qO- 

revids = [int(line) for line in revids_f[1:]]
len(revids)

20000

OK.  Now that we have a set of revisions, we need to label them.  In this case, we're going to label them as reverted/not.  We want to exclude a few different types of reverts -- e.g. when a user reverts themself or when an edit is reverted back to by someone else.  For this, we'll use the [mwreverts](https://pythonhosted.org/mwreverts) and [mwapi](https://pythonhosted.org/mwapi) libraries.  

In [3]:
import sys, traceback
import mwreverts.api
import mwapi

# We'll use the mwreverts API check.  In order to do that, we need an API session
session = mwapi.Session("https://en.wikipedia.org", 
                        user_agent="Revert detection demo <ahalfaker@wikimedia.org>")

# For each revision, find out if it was "reverted" and label it so.
rev_reverteds = []
for rev_id in revids[:20]:  # TODO: Limiting to the first 20!!!!
    try:
        _, reverted, reverted_to = mwreverts.api.check(
            session, rev_id, radius=5,  # most reverts within 5 edits
            window=48*60*60,  # 2 days
            rvprop={'user', 'ids'})  # Some properties we'll make use of
    except RuntimeError as e:
        sys.stderr.write(str(e))
        continue
    
    if reverted is not None:
        reverted_doc = [r for r in reverted.reverteds
                        if r['revid'] == rev_id][0]

        # self-reverts
        self_revert = \
            reverted_doc['user'] == reverted.reverting['user']
        
        # revisions that are reverted back to by others
        reverted_back_to = \
            reverted_to is not None and \
            reverted_doc['user'] != \
            reverted_to.reverting['user']
        
        # If we are reverted, not by self or reverted back to by someone else, 
        # then, let's assume it was damaging.
        damaging_reverted = !(self_revert or reverted_back_to)
    else:
        damaging_reverted = False

    rev_reverteds.append((rev_id, damaging_reverted))
    sys.stderr.write("r" if damaging_reverted else ".")


...............r....

Eeek!  This takes too long.  You get the idea.  So, I uploaded dataset that has already been labeled here @ `../datasets/demo/enwiki.rev_reverted.20k_2015.tsv.bz2`

In [4]:
rev_reverteds_f = !bzcat ../datasets/demo/enwiki.rev_reverted.20k_2015.tsv.bz2
rev_reverteds = [line.strip().split("\t") for line in rev_reverteds_f[1:]]
rev_reverteds = [(int(rev_id), reverted == "True") for rev_id, reverted in rev_reverteds]
len(rev_reverteds)

19868

OK.  It looks like we got an error when trying to extract the reverted status of ~132 edits, which is an acceptable loss.  Now just to make sure we haven't gone crazy, let's check some of the reverted edits:

* https://en.wikipedia.org/wiki/?diff=695071713 (section blanking)
* https://en.wikipedia.org/wiki/?diff=667375206 (unexplained addition of nonsense)
* https://en.wikipedia.org/wiki/?diff=670204366 (vandalism "I don't know")
* https://en.wikipedia.org/wiki/?diff=680329354 (adds non-existent category)
* https://en.wikipedia.org/wiki/?diff=668682186 (test edit -- removes punctuation)
* https://en.wikipedia.org/wiki/?diff=666882037 (adds spamlink)
* https://en.wikipedia.org/wiki/?diff=663302354 (adds nonsense special char)
* https://en.wikipedia.org/wiki/?diff=675803278 (unconstructive link changes)
* https://en.wikipedia.org/wiki/?diff=680203994 (vandalism -- "Pepe meme")
* https://en.wikipedia.org/wiki/?diff=656734057 ("JELENAS BOOTY UNDSO")

OK.  Looks like we are doing pretty good. :) 

## Part 2: Split the data into a training and testing set
<img style="float: right; margin: 1ex;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Machine_learning_nutshell_--_Split_into_train-test_set.svg/320px-Machine_learning_nutshell_--_Split_into_train-test_set.svg.png" />
Before we move on with training, it's important that we hold back some of the data for testing later.  If we train on the same data we'll test with, we risk [overfitting](https://en.wikipedia.org/wiki/Overfitting) and not noticing!

In this section, we'll both split the training and testing set *and* gather prective features for each of the labeled observations.

In [9]:
train_set = rev_reverteds[:15000]
test_set = rev_reverteds[15000:]
print("training:", len(train_set))
print("testing:", len(test_set))

training: 15000
testing: 4868


OK.  In order to train the machine learning model, we'll need to give it a source of signal.  This is where "features" come into play.  A feature represents a simple numerical statistic that we can extract from our observations that we think will be *predictive* of our outcome.  Luckily, `revscoring` provides a whole suite of features that work well for damage detection.  In this case, we'll be looking at features of the edit diff.  

In [11]:
from revscoring.features import wikitext, revision_oriented, temporal
from revscoring.languages import english

features = [
    # Catches long key mashes like kkkkkkkkkkkk
    wikitext.revision.diff.longest_repeated_char_added,
    # Measures the size of the change in added words
    wikitext.revision.diff.words_added,
    # Measures the size of the change in removed words
    wikitext.revision.diff.words_removed,
    # Measures the proportional change in "badwords"
    english.badwords.revision.diff.match_prop_delta_sum,
    # Measures the proportional change in "informals"
    english.informals.revision.diff.match_prop_delta_sum,
    # Measures the proportional change meaningful words
    english.stopwords.revision.diff.non_stopword_prop_delta_sum,
    # Is the user anonymous
    revision_oriented.revision.user.is_anon,
    # Is the user a bot or a sysop
    revision_oriented.revision.user.in_group({'bot', 'sysop'}),
    # How long ago did the user register?
    temporal.revision.user.seconds_since_registration
]

Now, we'll need to turn to `revscoring`s feature extractor to help us get us feature values for each revision.

In [13]:
from revscoring.extractors import api
api_extractor = api.Extractor(session)

print("rev_id:", 695071713)
print("\tfeatures:", list(api_extractor.extract(695071713, features)))
print("rev_id:", 667375206)
print("\tfeatures:", list(api_extractor.extract(667375206, features)))

rev_id: 695071713
	features: [1, 0, 10974, -1.0, -2.547619047619048, -1477.9699604325458, True, False, 313948852]
rev_id: 667375206
	features: [1, 1, 1, 0.0, 0.0, 0.33333333333333337, False, False, 9844289]


In [14]:
# Now for the whole set!
training_features_reverted = []
for rev_id, reverted in train_set[:20]:
    try:
        feature_values = list(api_extractor.extract(rev_id, features))
    except RuntimeError as e:
        sys.stderr.write(str(e))
        continue
    
    sys.stderr.write(".")
    training_features_reverted.append((rev_id, feature_values, reverted))
    

....................

Eeek!  Again this takes too long, so again, I uploaded a dataset with features already extracted @ `../datasets/demo/enwiki.features_reverted.training.20k_2015.tsv.bz2`

In [15]:
from revscoring.utilities.util import read_observations
training_features_reverted_f = !bzcat ../datasets/demo/enwiki.features_reverted.training.20k_2015.tsv.bz2 | cut -f2-
training_features_reverted = list(read_observations(training_features_reverted_f, features, lambda v: v=="True"))
len(training_features_reverted)

14979

## Part 3: Training the model
<img style="float: right; margin: 1ex;" src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/7a/Machine_learning_nutshell_--_Train_a_machine_learning_model.svg/320px-Machine_learning_nutshell_--_Train_a_machine_learning_model.svg.png" />

Now that we have a set of features extracted for our training set, it's time to train a model.  `revscoring` provides a set of different classifier algorithms.  From past experience, I know a [gradient boosting](https://en.wikipedia.org/wiki/Gradient_boosting) classifier works well, so we'll use that.  

In [16]:
from revscoring.scorer_models import GradientBoosting
is_reverted = GradientBoosting(features, version="live demo!", 
                               learning_rate=0.01, max_features="log2", 
                               n_estimators=700, max_depth=5,
                               balanced_sample_weight=True, scale=True, center=True)

is_reverted.train(training_features_reverted)

{'seconds_elapsed': 15.805180072784424}

We now have a trained model that we can play around with.  Let's try a few edits from our test set.

In [17]:
reverted_obs = [rev_id for rev_id, reverted in test_set if reverted]
non_reverted_obs = [rev_id for rev_id, reverted in test_set if not reverted]

for rev_id in reverted_obs[:10]:
    feature_values = list(api_extractor.extract(rev_id, features))
    score = is_reverted.score(feature_values)
    print(True, "https://en.wikipedia.org/wiki/?diff=" + str(rev_id), 
          score['prediction'], round(score['probability'][True], 2))

for rev_id in non_reverted_obs[:10]:
    feature_values = list(api_extractor.extract(rev_id, features))
    score = is_reverted.score(feature_values)
    print(False, "https://en.wikipedia.org/wiki/?diff=" + str(rev_id), 
          score['prediction'], round(score['probability'][True], 2))

True https://en.wikipedia.org/wiki/?diff=699665317 True 0.83
True https://en.wikipedia.org/wiki/?diff=683832871 True 0.81
True https://en.wikipedia.org/wiki/?diff=653913156 True 0.7
True https://en.wikipedia.org/wiki/?diff=654545786 True 0.82
True https://en.wikipedia.org/wiki/?diff=670608733 True 0.78
True https://en.wikipedia.org/wiki/?diff=689399141 True 0.71
True https://en.wikipedia.org/wiki/?diff=662365029 True 0.9
True https://en.wikipedia.org/wiki/?diff=656782076 True 0.87
True https://en.wikipedia.org/wiki/?diff=698954388 True 0.85
True https://en.wikipedia.org/wiki/?diff=645603577 True 0.67
False https://en.wikipedia.org/wiki/?diff=687073859 False 0.38
False https://en.wikipedia.org/wiki/?diff=665341163 False 0.15
False https://en.wikipedia.org/wiki/?diff=654524549 False 0.06
False https://en.wikipedia.org/wiki/?diff=682425664 False 0.07
False https://en.wikipedia.org/wiki/?diff=674780271 False 0.26
False https://en.wikipedia.org/wiki/?diff=684793059 False 0.08
False https://

## Part 4: Testing the model
So, the above analysis can help give us a sense for whether the model is working or not, but it's hard to standardize between models.  So, we can apply some metrics that are specially crafted for machine learning models.  

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/49/Machine_learning_nutshell_--_Test_the_machine_learning_model.svg/640px-Machine_learning_nutshell_--_Test_the_machine_learning_model.svg.png" />
</center>

But first, I'll need to load the pre-generated feature values.

In [18]:
testing_features_reverted_f = !bzcat ../datasets/demo/enwiki.features_reverted.testing.20k_2015.tsv.bz2 | cut -f2-
testing_features_reverted = list(read_observations(testing_features_reverted_f, features, lambda v: v=="True"))
len(testing_features_reverted)

4862

* [Accuracy](https://en.wikipedia.org/wiki/Accuracy_and_precision) -- The proportion of correct predictions
* [Precision](https://en.wikipedia.org/wiki/Precision_and_recall) -- The proportion of correct positive predictions
* [Recall](https://en.wikipedia.org/wiki/Precision_and_recall) -- The proportion of positive examples predicted as positive
* Filter rate at 90% recall -- The proportion of observations that can be ignored while still catching 90% of "reverted" edits.  

We'll use `revscoring` statistics to measure these against the test set.

In [19]:
from revscoring.scorer_models.test_statistics import (accuracy, precision, recall, 
                                                      roc, filter_rate_at_recall)

is_reverted.test(testing_features_reverted, 
                 test_statistics=[accuracy(), precision(), recall(), roc(), 
                                  filter_rate_at_recall(0.90)])

print(is_reverted.format_info())

ScikitLearnClassifier
 - type: GradientBoosting
 - params: balanced_sample_weight=true, presort="auto", n_estimators=700, min_weight_fraction_leaf=0.0, max_leaf_nodes=null, learning_rate=0.01, balanced_sample=false, init=null, random_state=null, warm_start=false, min_samples_leaf=1, scale=true, subsample=1.0, min_samples_split=2, verbose=0, center=true, max_depth=5, max_features="log2", loss="deviance"
 - version: live demo!
 - trained: 2016-04-06T09:24:54.429304

Accuracy: 0.813
Precision: 0.201
Recall: 0.812
ROC-AUC: 0.87
Filter rate @ 0.9 recall: threshold=0.238, filter_rate=0.651, recall=0.906


# Bonus round!  Let's listen to Wikipedia's vandalism!

So we don't have the most powerful damage detection classifier, but then again, we're only including 9 features.  Usually we run with ~60 features and get to much higher levels of fitness.  *but* this model is still useful and it should help us detect the most aggregious vandalism in Wikipedia.  In order to listen to Wikipedia, we'll need to connect to [RCStream](https://wikitech.wikimedia.org/wiki/RCStream) -- the same live feed that powers [listen to Wikipedia](http://listen.hatnote.com/).

In [20]:
import socketIO_client

class WikiNamespace(socketIO_client.BaseNamespace):
    def on_change(self, change):
        if change['type'] not in ('new', 'edit'):
            return
        
        rev_id = change['revision']['new']
        feature_values = list(api_extractor.extract(rev_id, features))
        score = is_reverted.score(feature_values)
        if score['prediction']:
            print("!!!Please review", "https://en.wikipedia.org/wiki/?diff=" + str(rev_id), 
                  round(score['probability'][True], 2), flush=True)
        else:
            print("Good edit", "https://en.wikipedia.org/wiki/?diff=" + str(rev_id),
                  round(score['probability'][True], 2), flush=True)

    def on_connect(self):
        self.emit('subscribe', 'en.wikipedia.org')


socketIO = socketIO_client.SocketIO('stream.wikimedia.org', 80)
socketIO.define(WikiNamespace, '/rc')

socketIO.wait(120)

Good edit https://en.wikipedia.org/wiki/?diff=713913478 0.17
Good edit https://en.wikipedia.org/wiki/?diff=713913481 0.24
Good edit https://en.wikipedia.org/wiki/?diff=713913480 0.16
Good edit https://en.wikipedia.org/wiki/?diff=713913483 0.15
Good edit https://en.wikipedia.org/wiki/?diff=713913479 0.26
Good edit https://en.wikipedia.org/wiki/?diff=713913482 0.12
!!!Please review https://en.wikipedia.org/wiki/?diff=713913484 0.88
Good edit https://en.wikipedia.org/wiki/?diff=713913485 0.17
Good edit https://en.wikipedia.org/wiki/?diff=713913487 0.21
Good edit https://en.wikipedia.org/wiki/?diff=713913486 0.09
Good edit https://en.wikipedia.org/wiki/?diff=713913488 0.06
!!!Please review https://en.wikipedia.org/wiki/?diff=713913489 0.55
Good edit https://en.wikipedia.org/wiki/?diff=713913490 0.14
!!!Please review https://en.wikipedia.org/wiki/?diff=713913491 0.6
!!!Please review https://en.wikipedia.org/wiki/?diff=713913492 0.62
Good edit https://en.wikipedia.org/wiki/?diff=713913493 0.

Good edit https://en.wikipedia.org/wiki/?diff=713913523 0.11
Good edit https://en.wikipedia.org/wiki/?diff=713913524 0.04
Good edit https://en.wikipedia.org/wiki/?diff=713913520 0.23
!!!Please review https://en.wikipedia.org/wiki/?diff=713913526 0.55
Good edit https://en.wikipedia.org/wiki/?diff=713913525 0.33
Good edit https://en.wikipedia.org/wiki/?diff=713913527 0.05
!!!Please review https://en.wikipedia.org/wiki/?diff=713913528 0.52
Good edit https://en.wikipedia.org/wiki/?diff=713913529 0.37
Good edit https://en.wikipedia.org/wiki/?diff=713913530 0.19
!!!Please review https://en.wikipedia.org/wiki/?diff=713913531 0.56
Good edit https://en.wikipedia.org/wiki/?diff=713913532 0.24
Good edit https://en.wikipedia.org/wiki/?diff=713913533 0.14
Good edit https://en.wikipedia.org/wiki/?diff=713913535 0.17
Good edit https://en.wikipedia.org/wiki/?diff=713913534 0.15
Good edit https://en.wikipedia.org/wiki/?diff=713913537 0.21
Good edit https://en.wikipedia.org/wiki/?diff=713913536 0.23
Goo

Good edit https://en.wikipedia.org/wiki/?diff=713913584 0.25
Good edit https://en.wikipedia.org/wiki/?diff=713913586 0.31
!!!Please review https://en.wikipedia.org/wiki/?diff=713913587 0.72
Good edit https://en.wikipedia.org/wiki/?diff=713913585 0.04
Good edit https://en.wikipedia.org/wiki/?diff=713913588 0.18
Good edit https://en.wikipedia.org/wiki/?diff=713913589 0.2
Good edit https://en.wikipedia.org/wiki/?diff=713913590 0.23
Good edit https://en.wikipedia.org/wiki/?diff=713913591 0.31
Good edit https://en.wikipedia.org/wiki/?diff=713913592 0.14
Good edit https://en.wikipedia.org/wiki/?diff=713913593 0.36
Good edit https://en.wikipedia.org/wiki/?diff=713913595 0.16
Good edit https://en.wikipedia.org/wiki/?diff=713913594 0.17
!!!Please review https://en.wikipedia.org/wiki/?diff=713913597 0.63
!!!Please review https://en.wikipedia.org/wiki/?diff=713913598 0.7
Good edit https://en.wikipedia.org/wiki/?diff=713913599 0.13
!!!Please review https://en.wikipedia.org/wiki/?diff=713913600 0.5

Good edit https://en.wikipedia.org/wiki/?diff=713913628 0.14
Good edit https://en.wikipedia.org/wiki/?diff=713913630 0.06
!!!Please review https://en.wikipedia.org/wiki/?diff=713913631 0.76
Good edit https://en.wikipedia.org/wiki/?diff=713913632 0.23
Good edit https://en.wikipedia.org/wiki/?diff=713913634 0.03
Good edit https://en.wikipedia.org/wiki/?diff=713913636 0.12
Good edit https://en.wikipedia.org/wiki/?diff=713913635 0.16
Good edit https://en.wikipedia.org/wiki/?diff=713913633 0.16
Good edit https://en.wikipedia.org/wiki/?diff=713913639 0.09
!!!Please review https://en.wikipedia.org/wiki/?diff=713913640 0.7
Good edit https://en.wikipedia.org/wiki/?diff=713913638 0.14
Good edit https://en.wikipedia.org/wiki/?diff=713913637 0.13
Good edit https://en.wikipedia.org/wiki/?diff=713913641 0.2
Good edit https://en.wikipedia.org/wiki/?diff=713913642 0.09
!!!Please review https://en.wikipedia.org/wiki/?diff=713913644 0.59
Good edit https://en.wikipedia.org/wiki/?diff=713913643 0.1
Good e